In [1]:
import cv2
import mediapipe as mp
from math import hypot
import numpy as np
import pygetwindow as gw
import pyautogui
import time

In [ ]:
# --- MediaPipe setup ---
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=1
)
Draw = mp.solutions.drawing_utils

# --- Gesture thresholds (in pixels) -- 
up_val = 10    # below this = “pinch” → scroll down
down_val  = 10    # above this = “unpinch” → scroll up
DEBOUNCE_MS = 1000   # ignore new scrolls for this many ms

last_scroll_time = 0

# --- Camera setup ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Could not open webcam")

print("--- Main loop ---")


pts = []
try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(frameRGB)
        if result.multi_hand_landmarks:
            handlm = result.multi_hand_landmarks[0]
            h, w, _ = frame.shape
            for idx, lm in enumerate(handlm.landmark):
                x_px, y_px = int(lm.x * w), int(lm.y * h)
                pts.append((idx, x_px, y_px,time.time()))
            Draw.draw_landmarks(frame, handlm, mpHands.HAND_CONNECTIONS)
        cv2.imshow('PDF Scroll Control', frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
finally:
    cap.release()
    cv2.destroyAllWindows()

--- Main loop ---


In [ ]:
np.savetxt('data.csv', pts, delimiter=',')